<a href="https://colab.research.google.com/github/YousefAtefB/StanceCat-COV19/blob/main/bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing libraries

In [19]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/StanceCat-COV19

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/StanceCat-COV19


In [20]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import re
import nltk
from tqdm import tqdm
from sklearn.metrics import f1_score
nltk.download('punkt')
from torch.optim import Adam

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Data PreProcessing

In [21]:
train = pd.read_csv('Dataset/train.csv')
text_train, category_train, stance_train = train['text'], train['category'], train['stance']

dev = pd.read_csv('Dataset/dev.csv')
text_dev, category_dev, stance_dev = dev['text'], dev['category'], dev['stance']

In [22]:
train.head(10)

,text,category,stance
0,بيل غيتس يتلقى لقاح #كوفيد19 من غير تصوير الاب...,celebrity,1
1,وزير الصحة لحد اليوم وتحديدا هلأ بمؤتمروا الصح...,info_news,1
2,قولكن رح يكونو اد المسؤولية ب لبنان لما يوصل ...,info_news,1
3,#تركيا.. وزير الصحة فخر الدين قوجة يتلقى أول ج...,celebrity,1
4,وئام وهاب يشتم الدول الخليجية في كل طلة اعلامي...,personal,0
5,"لقاح #كورونا في أميركا.. قلق متزايد من ""التوزي...",info_news,0
6,لبنان اشترى مليونان لقاح امريكي اذا شلنا يلي ع...,info_news,1
7,من عوارض لقاح كورونا<LF>هو تهكير حسابك عتويتر<...,personal,0
8,هناك 1780 مليونيراً في لبنان. ماذا لو فُرضت ال...,unrelated,0
9,دعبول حضرتك منو انت وتطلب من قائد دولة إسلامية...,info_news,1


In [23]:
dev.head(10)

,text,category,stance
0,#مريم_رجوي: <LF>حظر خامنئي المجرم شراء #لقاح_ك...,info_news,1
1,#الصحة:<LF>•تم إعطاء 259.530 جرعة من لقاح #كور...,plan,1
2,#خادم_الحرمين - حفظه الله - يتلقى الجرعة الأول...,celebrity,1
3,#الصحه_العالميه: لقاحات #كورونا آمنة ولا خوف م...,info_news,1
4,"#وزيرة_الصحة ""#هالة_زايد"" تقول إنه يجرى مراجعة...",info_news,1
5,2️⃣ وانتهى الفريق من الدراسات قبل السريرية ونش...,info_news,1
6,عاجل 🔴 <LF>.<LF><LF>.<LF><LF>وزارة الصحة :<LF>...,plan,1
7,#فيديو | السفير الأميركي لدى السعودية بعد تلقي...,info_news,1
8,تصريحات وبس الحكومة مع السيسي علي حسب اللقطة! ...,info_news,0
9,الاتحاد الاوروبي تفاوض لشراء لقاحات الكورونا م...,info_news,1


In [24]:
text_train, category_train, stance_train = np.array(train['text']), np.array(train['category']), np.array(train['stance'])
text_dev, category_dev, stance_dev = np.array(dev['text']), np.array(dev['category']), np.array(dev['stance'])

print(text_train.shape, category_train.shape, stance_train.shape)
print(text_dev.shape, category_dev.shape, stance_dev.shape)

(6988,) (6988,) (6988,)
(1000,) (1000,) (1000,)


In [25]:
!pip install arabert
!pip install transformers
from arabert.preprocess import ArabertPreprocessor
from transformers import AutoTokenizer, AutoModelForMaskedLM

model_name="aubmindlab/bert-base-arabertv02-twitter"
arabert_prep = ArabertPreprocessor(model_name=model_name)

text = "ولن نبالغ إذا قلنا إن هاتف أو كمبيوتر المكتب في زمننا هذا ضروري"
arabert_prep.preprocess(text)
  
tokenizer = AutoTokenizer.from_pretrained("aubmindlab/bert-base-arabertv02-twitter")
model = AutoModelForMaskedLM.from_pretrained("aubmindlab/bert-base-arabertv02-twitter")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [26]:
example_text = text_train[0]
bert_input = tokenizer(example_text, padding='max_length', max_length = 100, truncation=True, return_tensors="pt")

print(bert_input['input_ids'])
print(bert_input['token_type_ids'])
print(bert_input['attention_mask'])

output = model(**bert_input)
print(output.logits.shape)

tensor([[    2,  6941, 28652, 16136, 34495,    10, 17448,   196, 13593,   306,
           650,  5166, 48342,   197,   139,   391,  5787,  7325,   197,   139,
           391, 11474,   139,  7829,   213,  3616,   185, 54749,   305,  2858,
         10639,   139,  1721,   338,  2779, 17613,  1686,   298,  5263,     1,
           634,   889, 13591,  5175,   323, 28892,    20,    20,    20,  9110,
           394,   418,  3682, 28892,  1422,   418,  6843, 36720,   306,  5263,
         62279,    77,   221,   221,   178,   223,    31,     1,     1,    89,
            20,    72,   219,     1,    54,   275,   272,   244,   244,   270,
           260,   264,   268,   179,     3,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0]])
tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [27]:
categories = set(category_train)
print(categories)
category2id = {word:i for i, word in enumerate(list(categories))}
print(category2id['celebrity'])

{'personal', 'advice', 'requests', 'unrelated', 'restrictions', 'others', 'info_news', 'plan', 'celebrity', 'rumors'}
8


# Model Building

## BERT

### Ideas to try
1) bi-directional
2) pre-training
3) multi-layers
4) BERT
5) transformers notebook
6) packed_padded_sequences
7) pre-trained embedding

### Building Model

In [28]:
class Dataset(torch.utils.data.Dataset):

  def __init__(self, x, y):

    x = x.copy()

    x = [arabert_prep.preprocess(text) for text in x]
    self.X = [tokenizer(text, padding='max_length', max_length = 512, truncation=True, return_tensors="pt") for text in x]

    self.Y = torch.tensor(y)

    self.len = len(x)

  def __len__(self):
    return self.len

  def __getitem__(self, idx):
    return self.X[idx], self.Y[idx]

In [29]:
stance_train_dataset = Dataset(text_train, stance_train + 1)
category_train_dataset = Dataset(text_train, [category2id[category] for category in category_train])

stance_dev_dataset = Dataset(text_dev, stance_dev + 1)
category_dev_dataset = Dataset(text_dev, [category2id[category] for category in category_dev])

In [30]:
from transformers import BertModel
class BertClassifier(nn.Module):

    def __init__(self, output_dim, dropout=0.5):

        super(BertClassifier, self).__init__()

        self.bert = BertModel.from_pretrained('aubmindlab/bert-base-arabertv02-twitter')
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, output_dim)
        self.relu = nn.ReLU()

    def forward(self, input_id, mask):

        _, pooled_output = self.bert(input_ids=input_id, attention_mask=mask, return_dict=False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        final_layer = self.relu(linear_output)

        return final_layer

In [31]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

### Training

In [32]:
def train(model, train_dataset, train_dataloader, criterion, optimizer):

  total_acc_train = 0
  total_loss_train = 0

  # for f1 score
  y_true, y_pred = [], []

  for train_input, train_label in tqdm(train_dataloader):

    # print("input_ids",train_input['input_ids'].shape)
    # print("attention_mask",train_input['attention_mask'].shape)

    train_label = train_label.to(device)
    mask = train_input['attention_mask'].to(device)
    input_id = train_input['input_ids'].squeeze(1).to(device)

    output = model(input_id, mask)

    # print(output.shape)

    batch_loss = criterion(output, train_label.long())
    total_loss_train += batch_loss.item()

    acc = (output.argmax(dim=1) == train_label).sum().item()
    total_acc_train += acc

    y_true += train_label.tolist() 
    y_pred += output.argmax(dim=1).tolist()

    model.zero_grad()
    batch_loss.backward()
    optimizer.step()

  # print(len(y_true), len(y_pred))
  f1_macro_train = f1_score(y_true, y_pred, average='macro')

  return total_loss_train/len(train_dataset), total_acc_train/len(train_dataset), f1_macro_train

In [33]:
def evaluate(model, test_dataset, test_dataloader, criterion):

  total_acc_test = 0
  total_loss_test = 0

  # for f1 score
  y_true, y_pred = [], []

  with torch.no_grad():

    for test_input, test_label in test_dataloader:

      test_label = test_label.to(device)

      # print(test_input['input_ids'].shape)
      # print("attention_mask",train_input['attention_mask'].shape)

      mask = test_input['attention_mask'].to(device)
      input_id = test_input['input_ids'].squeeze(1).to(device)

      output = model(input_id, mask)

      # print(output.shape)

      batch_loss = criterion(output, test_label.long())
      total_loss_test += batch_loss.item()

      acc = (output.argmax(dim=1) == test_label).sum().item()
      total_acc_test += acc

      y_true += test_label.tolist() 
      y_pred += output.argmax(dim=1).tolist()

  # print(len(y_true)-1, len(y_pred)-1)
  f1_macro_test = f1_score(y_true, y_pred, average='macro')

  return total_loss_test/len(test_dataset), total_acc_test/len(test_dataset), f1_macro_test

In [34]:
def train_evaluate(model, train_dataset, val_dataset, learning_rate, epochs, model_name):

  train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=2, shuffle=True)
  val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=2)

  use_cuda = torch.cuda.is_available()
  device = torch.device("cuda" if use_cuda else "cpu")

  criterion = nn.CrossEntropyLoss()
  optimizer = Adam(model.parameters(), lr= learning_rate)

  if use_cuda:
    model = model.cuda()
    criterion = criterion.cuda()

  best_f1_macro = 0

  for epoch_num in range(epochs):

    loss_train, acc_train, f1_macro_train = train(model, train_dataset, train_dataloader, criterion, optimizer)
    loss_val, acc_val, f1_macro_val = evaluate(model, val_dataset, val_dataloader, criterion)
          
    if f1_macro_val > best_f1_macro:
      best_f1_macro = f1_macro_val
    torch.save(model.state_dict(), f'Models/BERT/{model_name}/F1{f1_macro_val: .4f} Acc{acc_val: .4f}.pt')

    print(
        f'Epochs: {epoch_num + 1} \
        | Train Loss: {loss_train: .4f} \
        | Train Accuracy: {acc_train : .4f} \
        | Train F1_macro: {f1_macro_train: .4f} \
        | Val Loss: {loss_val: .4f} \
        | Val Accuracy: {acc_val: .4f} \
        | Val F1_macro: {f1_macro_val: .4f} \
        ')

In [35]:
EPOCHS = 5
OUTPUT_DIM = 3
LR = 1e-6

model = BertClassifier(OUTPUT_DIM)
              
train_evaluate(model, stance_train_dataset, stance_dev_dataset, LR, EPOCHS, 'Stance')

Some weights of the model checkpoint at aubmindlab/bert-base-arabertv02-twitter were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertModel were not initialized from the model checkpoint at aubmindlab/bert-base-arabertv02-twitter and are newly initialized: ['bert.pooler.dens

Epochs: 1         | Train Loss:  0.558         | Train Accuracy:  0.000         | Train F1_macro:  0.000         | Val Loss:  0.501         | Val Accuracy:  0.500         | Val F1_macro:  0.333         


100%|██████████| 1/1 [00:17<00:00, 17.14s/it]


Epochs: 2         | Train Loss:  0.549         | Train Accuracy:  0.000         | Train F1_macro:  0.000         | Val Loss:  0.540         | Val Accuracy:  0.500         | Val F1_macro:  0.333         


100%|██████████| 1/1 [00:13<00:00, 13.19s/it]


Epochs: 3         | Train Loss:  0.486         | Train Accuracy:  0.500         | Train F1_macro:  0.333         | Val Loss:  0.607         | Val Accuracy:  0.000         | Val F1_macro:  0.000         


100%|██████████| 1/1 [00:13<00:00, 13.86s/it]


Epochs: 4         | Train Loss:  0.555         | Train Accuracy:  0.000         | Train F1_macro:  0.000         | Val Loss:  0.563         | Val Accuracy:  0.500         | Val F1_macro:  0.333         


100%|██████████| 1/1 [00:14<00:00, 14.95s/it]


Epochs: 5         | Train Loss:  0.578         | Train Accuracy:  0.000         | Train F1_macro:  0.000         | Val Loss:  0.549         | Val Accuracy:  0.000         | Val F1_macro:  0.000         


In [36]:
EPOCHS = 5
OUTPUT_DIM = 10
LR = 1e-6

model = BertClassifier(OUTPUT_DIM)
              
train_evaluate(model, category_train_dataset, category_dev_dataset, LR, EPOCHS, 'Category')

Some weights of the model checkpoint at aubmindlab/bert-base-arabertv02-twitter were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertModel were not initialized from the model checkpoint at aubmindlab/bert-base-arabertv02-twitter and are newly initialized: ['bert.pooler.dens

Epochs: 1         | Train Loss:  1.228         | Train Accuracy:  0.000         | Train F1_macro:  0.000         | Val Loss:  1.141         | Val Accuracy:  0.000         | Val F1_macro:  0.000         


100%|██████████| 1/1 [00:29<00:00, 29.02s/it]


Epochs: 2         | Train Loss:  1.092         | Train Accuracy:  0.000         | Train F1_macro:  0.000         | Val Loss:  1.181         | Val Accuracy:  0.000         | Val F1_macro:  0.000         


100%|██████████| 1/1 [00:13<00:00, 13.02s/it]


Epochs: 3         | Train Loss:  1.174         | Train Accuracy:  0.000         | Train F1_macro:  0.000         | Val Loss:  1.251         | Val Accuracy:  0.000         | Val F1_macro:  0.000         


100%|██████████| 1/1 [00:18<00:00, 18.90s/it]


Epochs: 4         | Train Loss:  0.957         | Train Accuracy:  1.000         | Train F1_macro:  1.000         | Val Loss:  1.239         | Val Accuracy:  0.000         | Val F1_macro:  0.000         


100%|██████████| 1/1 [00:23<00:00, 23.03s/it]


Epochs: 5         | Train Loss:  1.052         | Train Accuracy:  0.000         | Train F1_macro:  0.000         | Val Loss:  1.196         | Val Accuracy:  0.000         | Val F1_macro:  0.000         
